In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# RandomForest implementation

**Brief specification:**
 - Use the base code below
 - In the `fit` method in the loop (`i` from 0 to `n_estimators-1`), fix the seed equal to (`random_state + i`). The idea is that at each iteration there's a new value of random seed to add more "randomness", but at the same time results are reproducible
 - After fixing the seed, select `max_features` features **without replacement**, save the list of selected feature ids in `self.feat_ids_by_tree`
 - Also make a bootstrap sample (i.e. **sampling with replacement**) of training instances. For that, resort to `np.random.choice` and its argument `replace`
 - Train a decision tree with specified (in a constructor) arguments `max_depth`, `max_features` and `random_state` (do not specify `class_weight`) on a corresponding subset of training data. 
 - The `fit` method returns the current instance of the class `RandomForestClassifierCustom`, that is `self`
 - In the `predict_proba` method, we need to loop through all the trees. For each prediction, obviously, we need to take only those features which we used for training the corresponding tree. The method returns predicted probabilities (`predict_proba`), averaged for all trees

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

In [3]:
class RandomForestClassifierCustom():
    def __init__(self, n_estimators=10, max_depth=10, max_features=10, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state

        self.trees = []
        self.feat_idxs_by_tree = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            
            # make it reproducible
            np.random.seed(self.random_state + i)

            # indexes of features to be used
            feat_idxs = np.random.choice(range(X.shape[1]), self.max_features, replace=False)

            # training instances
            examples = list(set(np.random.choice(range(X.shape[0]), X.shape[0], replace=True)))

            # append current idxs to remember them till later
            self.feat_idxs_by_tree.append(feat_idxs)

            # create a model
            m = DecisionTreeClassifier(max_depth=self.max_depth,
                                       max_features=self.max_features,
                                       random_state=self.random_state)

            # build a tree with current feature idxs
            m.fit(X[examples, :][:, feat_idxs], y[examples])

            # append current model (a built tree)
            self.trees.append(m)

        return self

    def predict_proba(self, X):
        pred_prob = []
        for i in range(self.n_estimators):
            feat_idxs = self.feat_idxs_by_tree[i]
            pred_prob.append(self.trees[i].predict_proba(X[:,feat_idxs]))
        return np.mean(pred_prob, axis=0)

Testing:

In [4]:
#?np.random.choice

In [5]:
np.random.choice(range(19), 10, replace=False)  # replace=False does not permit repetition

array([11, 15,  2,  8, 16,  1,  6, 10,  3, 18])

In [6]:
#np.random.choice(range(2499), 2499, replace=True)                 # array of random numbers from 0 to 2499; might be with repetition
#set(np.random.choice(range(2499), 2499, replace=True))            # ascendingly sorted; repeated values become one
list(set(np.random.choice(range(2499), 2499, replace=True)))[:10]  # make a list out of the precious set

[0, 2, 5, 6, 8, 9, 10, 12, 13, 14]

### Dataset

In [7]:
df = pd.read_csv('data/telecom_churn.csv', sep=',')
df.drop('State', axis=1, inplace=True)
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [8]:
df.shape

(3333, 19)

In [9]:
df.isna().sum()

Account length            0
Area code                 0
International plan        0
Voice mail plan           0
Number vmail messages     0
Total day minutes         0
Total day calls           0
Total day charge          0
Total eve minutes         0
Total eve calls           0
Total eve charge          0
Total night minutes       0
Total night calls         0
Total night charge        0
Total intl minutes        0
Total intl calls          0
Total intl charge         0
Customer service calls    0
Churn                     0
dtype: int64

In [10]:
df['International plan'] = df['International plan'].map(dict(Yes=1, No=0))
df['Voice mail plan'] = df['Voice mail plan'].map(dict(Yes=1, No=0))
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', axis=1), df.Churn)

In [12]:
X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

(2499, 834, 2499, 834)

### Let's try to find best params

In [13]:
max_depth_values = range(5, 15)
max_features_values = [4, 5, 6, 7]
forest_params = {'max_depth': max_depth_values,
                 'max_features': max_features_values}

In [14]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [15]:
rf = RandomForestClassifier()
rf_grid_search = GridSearchCV(rf, forest_params, n_jobs=-1, scoring='accuracy', cv=skf)
rf_grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(5, 15), 'max_features': [4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [16]:
rf_grid_search.best_score_

0.9479791916766707

In [17]:
rf_grid_search.best_params_

{'max_depth': 10, 'max_features': 6}

In [18]:
m_d = rf_grid_search.best_params_['max_depth']
m_f = rf_grid_search.best_params_['max_features']
m_d, m_f

(10, 6)

### Custom RF

In [19]:
m = RandomForestClassifierCustom(max_depth=m_d, max_features=m_f)
m.fit(X_train.values, y_train.values)

In [20]:
# test to see what probs are for 1 and 0
# m.predict_proba(X_train.values)

#  > array([[0.96071828, 0.03928172],
#  >        [0.34883578, 0.65116422],
#  >        [0.82097145, 0.17902855], ...

# in the above, the first result is probably 0, second is 1
# y_train.head(3)  # False, True, False

In [21]:
pred_proba = m.predict_proba(X_test.values)
pred_proba

array([[0.84340249, 0.15659751],
       [0.8733407 , 0.1266593 ],
       [0.496     , 0.504     ],
       ...,
       [0.9661983 , 0.0338017 ],
       [0.93980764, 0.06019236],
       [0.97789151, 0.02210849]])

In [22]:
# list of predictions
y_pred = []
for i in pred_proba:
    if i[0] < i[1]:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [23]:
accuracy_score(y_test, pd.Series(y_pred))

0.8920863309352518

### Real RF

In [26]:
rf = RandomForestClassifier(max_depth=m_d, max_features=m_f)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9568345323741008